In [80]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pymongo

In [76]:
cols = ['name', 'date', 'address', 'infoLink']
catagoryIndexDict = {'2': 'Learning', '4': 'Performance', '11': 'Exhibition', '7': 'Mixer', '8': 'Foods', '6': 'Outdoors'}
catagoryValueDict = {'Learning': None, 'Performance': None, 'Exhibition': None, 'Mixer': None, 'Outdoors': None}

In [77]:
def parseEventData(urlBase, catagory):
    catagoryValue = catagoryIndexDict[catagory]
    pageUrl = ''
    maxPage = 30
    
    df_name = pd.DataFrame(columns = [cols[0]])
    df_date = pd.DataFrame(columns = [cols[1]])
    df_address = pd.DataFrame(columns = [cols[2]])
    df_info = pd.DataFrame(columns = [cols[3]])
    print('Running catagory:', catagoryValue)
    for page in range(2, maxPage):
        res = requests.get(urlBase + catagory + pageUrl)         
        soup = soup = BeautifulSoup(res.text, "lxml")

        rawEventNameData = soup.select('h2 a')
        rawInfoHref = soup.select('.btn-point')

        for div in soup.find_all('div', attrs={'class': 'date'}):
            timeSlot = div.get_text()
            df_date = df_date.append({'date': timeSlot}, ignore_index=True)

        for event in rawEventNameData:
            eventName = event.get_text()
            df_name = df_name.append({'name': eventName}, ignore_index=True)

        for a in soup.find_all('a', href=True, attrs={'class': 'btn-point btn-small pull-right'}):
            eventInfoLink = a['href']
            df_info = df_info.append({'infoLink': eventInfoLink}, ignore_index=True)

            infoRes = requests.get(eventInfoLink)
            infoSoup = BeautifulSoup(infoRes.text, "lxml")
            for span in infoSoup.find_all('span', attrs={'class': 'info-desc'}):
                if span.findChildren('i', attrs={'class': 'fa fa-map-marker'}):
                    address = span.get_text()
                    df_address = df_address.append({'address': address}, ignore_index=True)
        pageUrl = '&page=' + str(page)
        print('Running page:', page)
                    
    df = pd.concat([df_name, df_date, df_address, df_info], axis=1)
    df = df.replace(r'^\\s*$', np.nan, regex=True)
    df = df.dropna()
    df = df.to_dict('records')
    catagoryValueDict[catagoryValue] = df

In [78]:
catagories = ['2', '4', '11', '7', '8' ,'6']

for catagory in catagories:
    urlBased = 'https://kktix.com/events?category_id='
    parseEventData(urlBased, catagory)

Running catagory: Learning
Running page: 2
Running page: 3
Running page: 4
Running page: 5
Running page: 6
Running page: 7
Running page: 8
Running page: 9
Running page: 10
Running page: 11
Running page: 12
Running page: 13
Running page: 14
Running page: 15
Running page: 16
Running page: 17
Running page: 18
Running page: 19
Running page: 20
Running page: 21
Running page: 22
Running page: 23
Running page: 24
Running page: 25
Running page: 26
Running page: 27
Running page: 28
Running page: 29
Running catagory: Performance
Running page: 2
Running page: 3
Running page: 4
Running page: 5
Running page: 6
Running page: 7
Running page: 8
Running page: 9
Running page: 10
Running page: 11
Running page: 12
Running page: 13
Running page: 14
Running page: 15
Running page: 16
Running page: 17
Running page: 18
Running page: 19
Running page: 20
Running page: 21
Running page: 22
Running page: 23
Running page: 24
Running page: 25
Running page: 26
Running page: 27
Running page: 28
Running page: 29
Running

In [89]:
print(catagoryValueDict['Learning'])
uri = "mongodb://hackathon-db:ZioX47Y090fuSWSImTmFgoFfw1vYw9nltSLQneb4Smzw9p1dSvP113o1SFI7N8k0I3VjhBGDyR3p2odNkXKE0A==@hackathon-db.documents.azure.com:10255/?ssl=true&replicaSet=globaldb"
client = pymongo.MongoClient(uri)

[{'name': '吸引人的說話技巧～小蛙老師的自在表達課', 'date': '2019/05/14 19:00(+0800)', 'address': '衍聲說藝教室 / 台北市中正區新生南路一段50-1號7樓', 'infoLink': 'https://naturalvoices.kktix.cc/events/20190514'}, {'name': '[ 2019.04.13 開課！]【寫作小學堂】～寫出專屬風格，找回文字悸動~第二期', 'date': '2019/04/13 19:00(+0800)', 'address': '啟點文化~ / 台北市中正區博愛路52號9樓 ', 'infoLink': 'https://turn-key.kktix.cc/events/koob79'}, {'name': '不費力的發聲技巧～小蛙老師的輕鬆說話課', 'date': '2019/03/13 19:00(+0800)', 'address': '衍聲說藝坊教室 / 台北市中正區新生南路一段50-1號7樓', 'infoLink': 'https://naturalvoices.kktix.cc/events/20190313'}, {'name': '[2019.03.08 開課！] 《學「問」～高難度對話的望聞問切》~第15期', 'date': '2019/03/08 19:00(+0800)', 'address': '啟點文化~ / 台北市中正區博愛路52號9樓 ', 'infoLink': 'https://turn-key.kktix.cc/events/koob78'}, {'name': '電子電路+MCU單晶片韌體設計 — ARM Cortex M微控制器培訓基地【免費諮詢】', 'date': '2019/01/20 09:30(+0800)~17:30', 'address': '艾鍗學院 / 台北市中正區重慶南路一段143號4樓', 'infoLink': 'https://ittraining.kktix.cc/events/mcuplus'}, {'name': '健美行動小學堂愛分享系列講座-1月愛健康主題：育兒好好玩', 'date': '2019/01/12 14:30(+0800)~16:00', 'address

In [90]:
database = client['Events']
for key in catagoryValueDict.keys():
    collection = database[key]
    collection.insert_many(catagoryValueDict[key])